In [76]:
!pip install nltk
!pip install pickle
!pip install xgboost
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
#from wordcloud import WordCloud
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

# Load your data (replace with your actual data loading)
# Assuming your data is in a CSV file named 'data.csv'
import pandas as pd
new_columns=pd.read_csv('azureml://subscriptions/512a781e-d15a-4734-adca-96ec827531cb/resourcegroups/xwang3306-rg/workspaces/DVA_PROJECT/datastores/workspaceblobstore/paths/UI/2024-11-05_152711_UTC/neiss_10p_sample_new_columns.csv')
data_10P = pd.read_csv('../xwang3306/NEISS_10P_Sample.csv')
sematic=pd.read_csv('../xwang3306/10P_sematic_bert.csv')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[nltk_data] Downloading pack

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [2]:
#data_10p=data.sample(frac=0.1,random_state=42).reset_index(drop=True)
#data_10p.to_csv('NEISS_10P_Sample.csv')

In [3]:
data=data_10P.merge(new_columns,how='inner',on='CPSC_Case_Number').merge(sematic,how='inner',on='CPSC_Case_Number').reset_index(drop=True)
data.rename(columns={"sematic_distance_bert": "sematic_distance"},inplace=True)
data.head(5)

,Unnamed: 0,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Hispanic,Body_Part,Diagnosis,...,Stratum,PSU,Weight,Year,Month,Day,activity_at_injury,injury_mechanism,object_involved,sematic_distance
0,0,221032332,2022-09-24,14,1,0,0,0.0,34,71,...,V,77,17.2223,2022,9,24,playing basketball,fell,basketball,0.199045
1,1,181109464,2018-10-30,28,1,1,0,0.0,79,71,...,V,25,17.5136,2018,10,30,pick up crate,bent,crate,0.368001
2,2,210103105,2020-10-24,35,1,0,0,0.0,30,53,...,S,27,76.0369,2020,10,24,riding on mountain bike practicing,fell,mountain bike,0.162670
3,3,161157997,2016-11-15,214,2,0,0,0.0,76,53,...,S,48,85.2143,2016,11,15,coming down stairs,fell,stairs,0.150813
4,4,181107411,2018-10-21,4,1,0,0,0.0,92,72,...,C,20,4.9383,2018,10,21,playing with toy kitchen appliance,caught,toy kitchen appliance,0.274862


In [4]:
#data['sematic_distance']=sematic_distance

In [5]:
import numpy as np
data['#_prod']=np.nan
data.loc[((data['Product_1']>0)), '#_prod'] = 1
data.loc[((data['Product_1']>0) & (data['Product_2']>0)), '#_prod'] = 2
data.loc[((data['Product_1']>0) & (data['Product_2']>0) & (data['Product_3']>0)), '#_prod'] = 3

In [6]:
import numpy as np
data['Disposition_recode']=np.nan
data.loc[((data['Disposition']==1)), 'Disposition_recode'] = 0
data.loc[((data['Disposition']==2)), 'Disposition_recode'] = 1
data.loc[((data['Disposition']==4)), 'Disposition_recode'] = 2
data.loc[((data['Disposition']==5)), 'Disposition_recode'] = 3
data.loc[((data['Disposition']==8)), 'Disposition_recode'] = 4
#data=data[data['Disposition_recode'].notna()]

data['Disposition_recode_2']=0
data.loc[((data['Disposition_recode']>0)), 'Disposition_recode_2'] = 1

In [7]:
data['Disposition_recode'].value_counts()
data['Disposition_recode_2'].value_counts()

0    308854
1     37458
Name: Disposition_recode_2, dtype: int64

In [8]:
bdpt_dict={}
bdpt_dict[0]='INTERNAL'
bdpt_dict[30]='SHOULDER'
bdpt_dict[31]='UPPERTRUNK'
bdpt_dict[32]='ELBOW'
bdpt_dict[33]='LOWERARM'
bdpt_dict[34]='WRIST'
bdpt_dict[35]='KNEE'
bdpt_dict[36]='LOWERLEG'
bdpt_dict[37]='ANKLE'
bdpt_dict[38]='PUBICREGION'
bdpt_dict[75]='HEAD'
bdpt_dict[76]='FACE'
bdpt_dict[77]='EYEBALL'
bdpt_dict[78]='UPPERTRUNK(OLD)'
bdpt_dict[79]='LOWERTRUNK'
bdpt_dict[80]='UPPERARM'
bdpt_dict[81]='UPPERLEG'
bdpt_dict[82]='HAND'
bdpt_dict[83]='FOOT'
bdpt_dict[84]='25-50% OF BODY'
bdpt_dict[85]='ALLPARTSBODY'
bdpt_dict[86]='OTHER(OLD)'
bdpt_dict[87]='NOTSTATED/UNK'
bdpt_dict[88]='MOUTH'
bdpt_dict[89]='NECK'
bdpt_dict[90]='LOWERARM(OLD)'
bdpt_dict[91]='LOWERLEG(OLD)'
bdpt_dict[92]='FINGER'
bdpt_dict[93]='TOE'
bdpt_dict[94]='EAR'


In [9]:
data['body_string']=data['Body_Part'].map(bdpt_dict)

In [10]:
data['Narrative_LLM']=data["activity_at_injury"].astype(str) + ' '+data["injury_mechanism"].astype(str)+ ' ' + data["object_involved"].astype(str)

In [12]:
#data_2=data[['CPSC_Case_Number']].head(315900)
#data_2['sematic_distance_bert']=sematic_distance_bert
##data_2.to_csv('10P_sematic_bert_315900.csv',index=False)

In [13]:
#data['sematic_distance_bert']=sematic_distance_bert
#data_out=data[['CPSC_Case_Number','sematic_distance_bert']]
#data_out.to_csv('10P_sematic_bert.csv',index=False)

In [14]:
'''
import spacy
import requests

# Load the SpaCy model
nlp = spacy.load('en_core_web_md')

sematic_distance=[]

for i in range(len(data)):
    #print(i)
    word1 = data.at[i,'body_string'] #"cut"
    word2 = data.at[i,'Narrative_LLM'] #"bike"

    # SpaCy similarity
    doc1 = nlp(word1)
    doc2 = nlp(word2)
    similarity = doc1.similarity(doc2)
    sematic_distance.append(similarity)
'''

'\nimport spacy\nimport requests\n\n# Load the SpaCy model\nnlp = spacy.load(\'en_core_web_md\')\n\nsematic_distance=[]\n\nfor i in range(len(data)):\n    #print(i)\n    word1 = data.at[i,\'body_string\'] #"cut"\n    word2 = data.at[i,\'Narrative_LLM\'] #"bike"\n\n    # SpaCy similarity\n    doc1 = nlp(word1)\n    doc2 = nlp(word2)\n    similarity = doc1.similarity(doc2)\n    sematic_distance.append(similarity)\n'

In [77]:
def remove_after_dx(narrative):
  if isinstance(narrative, str):
    parts = narrative.split("DX", 1)
    if len(parts) > 1:
      return parts[0]
    else:
      return narrative  # No "DX:" found, return the original string
  else:
    return narrative  # Not a string, return as is

data['Narrative'] = data['Narrative'].apply(remove_after_dx)

In [78]:
import multiprocessing as mp
import re
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

medical_terms = {
    "&": "and",
    "***": "",
    ">>": "clinical diagnosis",
    "@": "at",
    "abd": "abdomen",
    "af": "accidental fall",
    "afib": "atrial fibrillation",
    "aki": "acute kidney injury",
    "am": "morning",
    "ams": "altered mental status",
    "bac": "blood alcohol content",
    "bal": "blood alcohol level,",
    "biba": "brought in by ambulance",
    "c/o": "complains of",
    "chi": "closed-head injury",
    "clsd": "closed",
    "cpk": "creatine phosphokinase",
    "cva": "cerebral vascular accident",
    "dx": "clinical diagnosis",
    "ecf": "extended-care facility",
    "er": "emergency room",
    "etoh": "ethyl alcohol",
    "eval": "evaluation",
    "fd": "fall detected",
    "fx": "fracture",
    "fxs": "fractures",
    "glf": "ground level fall",
    "h/o": "history of",
    "htn": "hypertension",
    "hx": "history of",
    "inj": "injury",
    "inr": "international normalized ratio",
    "intox": "intoxication",
    "l": "left",
    "loc": "loss of consciousness",
    "lt": "left",
    "mech": "mechanical",
    "mult": "multiple",
    "n.h.": "nursing home",
    "nh": "nursing home",
    "p/w": "presents with",
    "pm": "afternoon",
    "pt": "patient",
    "pta": "prior to arrival",
    "pts": "patient's",
    "px": "physical examination", # not "procedure",
    "r": "right",
    "r/o": "rules out",
    "rt": "right",
    "s'd&f": "slipped and fell",
    "s/p": "after",
    "sah": "subarachnoid hemorrhage",
    "sdh": "acute subdural hematoma",
    "sts": "sit-to-stand",
    "t'd&f": "tripped and fell",
    "tr": "trauma",
    "uti": "urinary tract infection",
    "w/": "with",
    "w/o": "without",
    "wks": "weeks"
}

# cleanning
def clean_narrative(text):
    # lowercase everything
    text = text.lower()
    
    # unglue DX
    regex_dx = r"([ˆ\W]*(dx)[ˆ\W]*)"
    text = re.sub(regex_dx, r". dx: ", text)

    # remove age and sex identifications
    ## regex to capture age and sex (not perfect but captures almost all of the cases)
    regex_age_sex = r"(\d+)\s*?(yof|yf|yo\s*female|yo\s*f|yom|ym|yr|yo\s*male|yo\s*m)"
    age_sex_match = re.search(regex_age_sex, text)

    ## format age and sex
    if age_sex_match:
        age = age_sex_match.group(1)
        sex = age_sex_match.group(2)
        
        # probably not best practice but it works with this data
        if "f" in sex:
            #text = text.replace(age_sex_match.group(0), f"{age} years old female")
            text = text.replace(age_sex_match.group(0), f"patient")
        elif "m" in sex:
            #text = text.replace(age_sex_match.group(0), f"{age} years old male")
            text = text.replace(age_sex_match.group(0), f"patient")
    
    # translate medical terms
    for term, replacement in medical_terms.items():
        if term == "@" or term == ">>" or term == "&" or term == "***":
            pattern = fr"({re.escape(term)})"
            text = re.sub(pattern, f" {replacement} ", text) # force spaces around replacement
            
        else:
            pattern = fr"(?<!-)\b({re.escape(term)})\b(?!-)"
            text = re.sub(pattern, replacement, text)

    # user-friendly format
    sentences = sent_tokenizer.tokenize(text)
    sentences = [sent.capitalize() for sent in sentences]
    return " ".join(sentences)

text = '72 YOF SLIPPED AND FELL ON THE FLOOR THIS AM. DX: L-3, L-4 FRACTURE, RIGHT RIB FRACTURES X 3.'
print("Original text:", text)
print("Clean text:", clean_narrative(text))
print()

# Use all CPU cores
with mp.Pool(mp.cpu_count()) as pool:
    data['Narrative'] = pool.map(clean_narrative, data['Narrative'])
    


Original text: 72 YOF SLIPPED AND FELL ON THE FLOOR THIS AM. DX: L-3, L-4 FRACTURE, RIGHT RIB FRACTURES X 3.
Clean text: Patient slipped and fell on the floor this morning. Clinical diagnosis: l-3, l-4 fracture, right rib fractures x 3.



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [72]:
data['Narrative']=data['Narrative'].str.upper()
data[['Narrative']]

,Narrative
0,S HE FALL 1 WEEK AND COMPLAINS OF HE HAS BE...
1,A BENT TO PICK UP CRATE AT HOME TO WITH LOW BACK
2,RIDING ON MOUNTAIN BIKE PRACTICING FALL DOWN
3,14 MONTH FE FOR AND WAS COMINGDOWN STAIRS...
4,4PLAYING WITH TOY KITCHEN APPLIANCE AND GOT ...
...,...
352046,HELPING LOAD BICYCLE IN VAN
352047,PUSHING DOWN FOOD IN BLEND MAKING A SMOOTHIE ...
352048,FALL DOWN STEPS
352049,WAS RIDING A BIKE AND FALL


In [79]:
replace_list=['ANKLE', 'ARM', 'BODY_PART', 'CHEST', 'CONTUSION', 'CUT', 'EAR', 'ELBOW', 'EYE', 'FACE', 'FINGER', 'FOOT', 'FOREHEAD', 'FRACTURE', 'FX', 'HAND', 'HEAD', 'HIP', 'KNEE', 'LAC', 'LACERATION', 'LEG', 'LOC', 'LOSE', 'NECK', 'PAIN', 'SHOULDER', 
'SPRAIN', 'STRAIN', 'SWELL', 'THUMB', 'TOE', 'WRIST','ABRASION', 'ACHE', 'BREAK', 'BURN', 'CHIN', 'CUT', 'ER', 'FRACTURE', 'FX', 'HIT', 'INJURY', 'LACERATION', 'LIP', 'LOSE', 'LOC', 'MOUTH', 'NOSE', 'PAIN', 'RIB', 'SCALP', 'SPRAIN', 'STRAIN', 'SWELL', 'TOE', 'TWIST', 'WRIST']

for i in replace_list:
  data['Narrative'] = data['Narrative'].str.replace(i, '')
  data['Narrative_LLM']= data['Narrative_LLM'].str.replace(i, '')


data['Narrative'] = data['Narrative'].str.replace('YOM', '')
data['Narrative'] = data['Narrative'].str.replace('YOF', '')
data['Narrative'] = data['Narrative'].str.replace('YR', '')
data['Narrative'] = data['Narrative'].str.replace('OLD', '')
data['Narrative'] = data['Narrative'].str.replace('MALE', '')
data['Narrative'] = data['Narrative'].str.replace('FEMALE', '')
data['Narrative'] = data['Narrative'].str.replace(' YO ', '')
data['Narrative'] = data['Narrative'].str.replace('YO ', '')
data['Narrative'] = data['Narrative'].str.replace(' F ', '')
data['Narrative'] = data['Narrative'].str.replace('YF', '')
data['Narrative'] = data['Narrative'].str.replace(' M ', '')
data['Narrative'] = data['Narrative'].str.replace('ACCIDENTALLY','')
data['Narrative'] = data['Narrative'].str.replace('AGO', '')
data['Narrative'] = data['Narrative'].str.replace('TODAY', '')
data['Narrative'] = data['Narrative'].str.replace('YESTERDAY', '')
data['Narrative'] = data['Narrative'].str.replace('PATIENT', '')
data['Narrative'] = data['Narrative'].str.replace(' PT ', '')
data['Narrative'] = data['Narrative'].str.replace('INJURY', '')
data['Narrative'] = data['Narrative'].str.replace('REPORT', '')
data['Narrative'] = data['Narrative'].str.replace('HURT', '')
data['Narrative'] = data['Narrative'].str.replace('INJ', '')
data['Narrative'] = data['Narrative'].str.replace('FELL', 'FALL')
data['Narrative'] = data['Narrative'].str.replace('INJURE', '')
data['Narrative'] = data['Narrative'].str.replace('JURED', '')
data['Narrative'] = data['Narrative'].str.replace('URED', '')
data['Narrative'] = data['Narrative'].str.replace(' ED', '')
data['Narrative'] = data['Narrative'].str.replace(' RT ', '')
data['Narrative'] = data['Narrative'].str.replace(' LT ', '')

data['Narrative_LLM'] = data['Narrative_LLM'].str.replace('fell', 'fall')


In [ ]:
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

revision = None  # Replace with the specific revision to ensure reproducibility if the model is updated.

model = SentenceTransformer("avsolatorio/GIST-small-Embedding-v0", revision=revision)

data=data.reset_index(drop=True)
gist_dict={}
for i in range(len(data)):    
    gist_dict[str(i)]=model.encode(data.at[i,'Narrative'], convert_to_tensor=True).numpy()

X_test=pd.DataFrame(gist_dict).T
out=pd.concat([data[['CPSC_Case_Number']],X_test],axis=1)
out.to_csv('gist_embedding_10p.csv',index=False)